In [8]:
import numpy as np
from scipy.sparse import diags, csr_matrix
from scipy.sparse.linalg import norm as sparse_norm
from numpy.linalg import norm
from scipy.optimize import minimize
from tensorflow.keras.datasets import mnist

# Load and preprocess MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28 * 28) / 255.0
x_test = x_test.reshape(-1, 28 * 28) / 255.0

# Select a subset of the data (e.g., first 1000 samples)
subset_size = 1000
x_subset = x_train[:subset_size]
y_subset = y_train[:subset_size]

# Define class assignment criteria
class_assignment = np.zeros(y_subset.shape[0])

for i, label in enumerate(y_subset):
    if label % 2 == 0:  # Even number
        class_assignment[i] = 1
    elif label % 2 == 1:  # Odd number
        class_assignment[i] = -1

# Construct the graph signal matrix S based on class assignment
S = class_assignment

# Reshape the images to 1D vectors
x_subset_1d = x_subset.reshape(-1, 28 * 28)

# Construct the adjacency matrix (1D chain graph) in sparse format
N = x_subset_1d.shape[0]  # Number of nodes
adjacency = diags([1], [1], shape=(N, N)) + diags([1], [-1], shape=(N, N))
adjacency = adjacency.tocsr()

# Normalize adjacency matrix
D_inv_sqrt = diags([1 / np.sqrt(adjacency.sum(axis=1)).A.flatten()], [0])
Anorm = D_inv_sqrt @ adjacency @ D_inv_sqrt

# Define the diagonal matrix C based on class assignment
C = np.diag(S)

# Define your optimization objective and gradient
def cost_function(S):
    CS_known = diagonal_matrix_C @ S
    CS_algorithm = C @ S
    diff_CS = CS_known - CS_algorithm
    J = 0.5 * (norm((S - (M1 + 2 * alpha * M2) @ S) @ Anorm)**2 +
               alpha * norm(diff_CS)**2)
    return J

def gradient(S):
    CS_known = diagonal_matrix_C @ S
    CS_algorithm = C @ S
    gradient_J = (M1 + 2 * alpha * M2) @ S - CS_known + CS_algorithm
    return gradient_J

# Define optimization function for scipy.optimize
def optimization_function(S):
    return cost_function(S), gradient(S)

# Set hyperparameters
alpha = 0.1
epsilon = 1e-6
max_iterations = 1000

# Precompute matrices M1 and M2
M1 = (np.eye(S.shape[0]) - Anorm).T @ (np.eye(S.shape[0]) - Anorm)
M2 = C.T @ C

# Define C matrix based on class assignment
C = np.diag(class_assignment)

# Use scipy.optimize to minimize the cost function
result = minimize(optimization_function, S, method='L-BFGS-B', jac=True,
                  options={'maxiter': max_iterations, 'gtol': epsilon})

# Extract the optimized signal matrix S
optimized_S = result.x

print("Final Optimized Signal Matrix S:")
print(optimized_S)


Final Optimized Signal Matrix S:
[ 1.85355652e-06  1.66316811e-06 -1.86565486e-06 -6.53096946e-06
 -9.65790405e-06 -8.52998503e-06 -2.65800140e-06  1.30652570e-06
  2.92428600e-06 -3.83201628e-07 -1.52366471e-06 -2.49798299e-06
 -3.75238300e-06 -5.99835401e-06 -6.44845875e-06 -2.73936842e-06
  7.12696673e-07  1.05977732e-06 -4.12383096e-07 -1.22685794e-07
  2.61260105e-06  2.74970607e-06 -2.72794373e-07 -4.17203448e-06
 -3.15214067e-06  1.24272657e-06  5.58992469e-06  6.05784914e-06
  3.05242816e-06  2.97339279e-07 -1.51464702e-06 -1.16008857e-06
 -2.06610767e-06 -1.41895074e-06 -5.39459281e-07  3.08407264e-06
  5.50595239e-06  4.47861896e-06  2.75135807e-07 -5.41834032e-06
 -8.13225181e-06 -7.44912735e-06 -4.59553761e-06 -9.22682198e-07
  1.12311310e-06  1.16129904e-07 -4.22686858e-06 -9.44572041e-06
 -1.09114521e-05 -7.04732520e-06 -1.83917900e-06  4.51908210e-07
 -2.60820609e-06 -6.59477958e-06 -4.66831823e-06  2.31188760e-06
  1.05902867e-05  1.10199504e-05  5.16449291e-06 -4.24607